Import keperluan

In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow import keras
from keras import layers, models

dataset_path = 'dataset/'



Membuat local variable untuk keperluan nantinya

In [2]:
# Prepare dataset
X = []
y = []

# Hanya untuk mengambil nama folder saja
folder_name = []


Logic untuk pemrorsesan dataset

In [3]:
# kita extract mfcc dari dataset kita
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, duration=2.5, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

# Mengambil seluruh data dari dataset yang kita miliki
for label, tajwid_category in enumerate(os.listdir(dataset_path)):
    category_path = os.path.join(dataset_path, tajwid_category)
    folder_name.append(tajwid_category)
    if os.path.isdir(category_path):
        for file in os.listdir(category_path):
            if file.endswith('.wav'):
                file_path = os.path.join(category_path, file)
                mfcc = extract_mfcc(file_path)
                X.append(mfcc)
                y.append(label)


# Ubah ke numpy array
X = np.array(X)
y = np.array(y)

Pembuatan Model

In [4]:
# Kita bagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# Kita compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Latih model
model.fit(X_train, y_train, epochs=1000, batch_size=2, validation_data=(X_test, y_test))

# evaluasi akurasi model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

d:\project\bu indah\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4652 - loss: 33.3507 - val_accuracy: 0.5000 - val_loss: 14.4590
Epoch 2/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1275 - loss: 20.0075 - val_accuracy: 0.2500 - val_loss: 6.3215
Epoch 3/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6135 - loss: 3.3250 - val_accuracy: 0.0000e+00 - val_loss: 11.7159
Epoch 4/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4017 - loss: 4.2196 - val_accuracy: 0.0000e+00 - val_loss: 3.8907
Epoch 5/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4049 - loss: 2.4927 - val_accuracy: 0.5000 - val_loss: 2.0603
Epoch 6/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5991 - loss: 1.6013 - val_accuracy: 0.2500 - val_loss: 6.9150
Epoch 7/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4987 - loss: 2.8112 - val_accuracy: 0.5000 - val_loss: 2.0408
Epoch 8/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5700 - loss: 1.1132 - val_accuracy: 0

Logic untuk melakukan pengetesan data

In [5]:
'''
Window size : Berguna untuk memotong suara menjadi bagian-bagian yang lebih kecil
Stride : merupakan jarak antara window 1 ke window yang lainnya
sr : merupakan sampe rate (22050 hz)
'''
def predict_long_audio(audio_path, window_size = 1.0, stride = 0.5, sr = 22050):
    y, _ = librosa.load(audio_path, sr=sr)
    duration = librosa.get_duration(y=y, sr=sr)
    
    predictions = []
    timestamps = []

    for start in np.arange(0, duration - window_size + stride, stride):
        end = start + window_size
        start_sample = int(start * sr)
        end_sample = int(end * sr)
        y_segment = y[start_sample:end_sample]

        if len(y_segment) < sr * window_size:
            padding = sr * window_size - len(y_segment)
            y_segment = np.pad(y_segment, (0, int(padding)))

        mfcc = librosa.feature.mfcc(y=y_segment, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc.T, axis=0).reshape(1, -1)

        prediction = model.predict(mfcc_mean)
        predicted_class = np.argmax(prediction)
        predictions.append(predicted_class)
        timestamps.append((round(start, 2), round(end, 2)))

    return list(zip(timestamps, predictions))

Melakukan pengetesan model

In [6]:
long_result = predict_long_audio('tes/tes_bismillah.mp3')
for (start, end), pred in long_result:
    print(f"{start}-{end} seconds: Tajwid {folder_name[pred]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
0.0-1.0 seconds: Tajwid idhzar
0.5-1.5 seconds: Tajwid idgham
1.0-2.0 seconds: Tajwid idgham
1.5-2.5 seconds: Tajwid idgham
2.0-3.0 seconds: Tajwid idgham
2.5-3.5 seconds: Tajwid idgham
3.0-4.0 seconds: Tajwid idgham
3.5-4.5 seconds: Tajwid idgham
4.0-5.0 seconds: Tajwid iqlab
4.5-5.5 seconds: Tajwid iqlab
5.0-6.0 seconds: Tajwid idhzar
5.5-6.5 seconds: Tajwid idhzar


Menyimpan model

In [7]:
# Menyimpan dengan model .h5, digunakan untuk pembuatan api dan pembacaan menggunakan api di client
model.save('tajwid_model.h5')
